In [34]:
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm_notebook

### Hyperparameter

In [6]:
EPOCHS = 5

### Network Structure

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = layers.Flatten(input_shape=(288,28))
        self.d1 = layers.Dense(128, activation='sigmoid')
        self.d2 = layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

### Train loop

In [27]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

### Test loop

In [28]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)  

### Preprocessing

In [29]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255., x_test/255.

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [30]:
# create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [32]:
# Define preformance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [37]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
        
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

Epoch 1, Loss: 0.15165694057941437, Accuracy: 95.8126220703125, Test Loss: 0.08256717026233673, Test Accuracy: 97.39582824707031
Epoch 2, Loss: 0.1389450579881668, Accuracy: 96.17041778564453, Test Loss: 0.07268574833869934, Test Accuracy: 97.76786041259766
Epoch 3, Loss: 0.12818977236747742, Accuracy: 96.4727783203125, Test Loss: 0.06955858319997787, Test Accuracy: 97.65625
Epoch 4, Loss: 0.11890670657157898, Accuracy: 96.73332977294922, Test Loss: 0.06411527842283249, Test Accuracy: 97.91667175292969
Epoch 5, Loss: 0.11080918461084366, Accuracy: 96.96651458740234, Test Loss: 0.05841786414384842, Test Accuracy: 98.125
